In [3]:
import os
from xml.etree import ElementTree
import numpy as np
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/HFtests/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

fatal: destination path 'TACL2015' already exists and is not an empty directory.
fatal: destination path 'nlu-asdiv-dataset' already exists and is not an empty directory.
fatal: destination path 'grade-school-math' already exists and is not an empty directory.


In [4]:
with open('/home/PracticalWork2021/data/TACL2015/questions.json', 'r') as f:
  data = json.load(f)

In [10]:
# Randomly choose a problem
rand_index = np.random.randint(0, len(data))
data[rand_index]

{'lEquations': ['2+7+1=1+x'],
 'lSolutions': [9.0],
 'grammarCheck': 0,
 'templateNumber': 0,
 'sQuestion': 'Zach wants to ride the Ferris wheel, the roller coaster, and the log ride. The Ferris wheel costs 2 tickets, the roller coaster costs 7 tickets and the log ride costs 1 ticket. Zach has 1 ticket. How many more tickets should Zach buy?',
 'iIndex': 278}